In [ ]:
function updateMonthlyOrdersSheet() {
  // Replace "123X" with the actual Google Sheet ID
  var sheet = SpreadsheetApp.openById("11REv4agFweH_w9JRkXdgI09auXdDeMoJSuvw8iI-Ihc").getSheetByName("Test");
  
  // Query to retrieve the data
  var query = `WITH
orders AS (
SELECT
client_id,
DATE_TRUNC(created_at, MONTH) AS month,
COUNT(DISTINCT order_id) AS order_count
FROM \`cb-prod-297913.cb_prod_analytics.cb_receive_trip_orders\`
WHERE created_at >= '2022-11-01'
AND NOT REGEXP_CONTAINS(branch.name, r'\bTest\b') 
AND NOT REGEXP_CONTAINS(branch.name, r'\bLoop\b')
AND NOT REGEXP_CONTAINS(branch.name, r'\Cowa-bunga\b')
AND NOT REGEXP_CONTAINS(branch.name, r'\Cowabunga\b')
AND branch.name IS NOT NULL
GROUP BY 1, 2
)
SELECT client_id,
SUM(COALESCE(CASE WHEN month = '2022-11-01' THEN order_count ELSE 0 END, 0)) AS Nov2022,
SUM(COALESCE(CASE WHEN month = '2022-12-01' THEN order_count ELSE 0 END, 0)) AS Dec2022,
SUM(COALESCE(CASE WHEN month = '2023-01-01' THEN order_count ELSE 0 END, 0)) AS Jan2023,
SUM(COALESCE(CASE WHEN month = '2023-02-01' THEN order_count ELSE 0 END, 0)) AS Feb2023,
SUM(COALESCE(CASE WHEN month = '2023-03-01' THEN order_count ELSE 0 END, 0)) AS Mar2023,
SUM(COALESCE(CASE WHEN month = '2023-04-01' THEN order_count ELSE 0 END, 0)) AS Apr2023,
SUM(COALESCE(CASE WHEN month = '2023-05-01' THEN order_count ELSE 0 END, 0)) AS May2023,
SUM(COALESCE(CASE WHEN month = '2023-06-01' THEN order_count ELSE 0 END, 0)) AS Jun2023,
SUM(COALESCE(CASE WHEN month = '2023-07-01' THEN order_count ELSE 0 END, 0)) AS Jul2023,
SUM(COALESCE(CASE WHEN month = '2023-08-01' THEN order_count ELSE 0 END, 0)) AS Aug2023,
SUM(COALESCE(CASE WHEN month = '2023-09-01' THEN order_count ELSE 0 END, 0)) AS Sep2023,
SUM(COALESCE(CASE WHEN month = '2023-10-01' THEN order_count ELSE 0 END, 0)) AS Oct2023,
SUM(COALESCE(CASE WHEN month = '2023-11-01' THEN order_count ELSE 0 END, 0)) AS Nov2023,
SUM(COALESCE(CASE WHEN month = '2023-12-01' THEN order_count ELSE 0 END, 0)) AS Dec2023
FROM orders
GROUP BY client_id`;
  
  // Run the query and retrieve the result
  var projectId = "cb-prod-297913";
  var request = {
    query: query,
    useLegacySql: false
  };
  var job = BigQuery.Jobs.query(request, projectId);
  var queryResults = BigQuery.Jobs.getQueryResults(projectId, job.jobReference.jobId);
  
  // Get the data and the column names from the result
  var data = queryResults.rows.map(function(row) { return row.f.map(function(cell) { return cell.v; }); });
  var columns = queryResults.schema.fields.map(function(field) { return field.name; });
  
  // Write the data and the column names to the sheet
  sheet.clearContents();
  sheet.getRange(1, 1, data.length + 1, data[0].length).setValues([columns].concat(data));
}
